In [ ]:
from pyopenms import *
import pandas as pd
import numpy as np


In [ ]:
# define input and output file locations
input_mgf = "/home/eugen/Development/EuBIC2020/data/clusters_maracluster.mgf"
output_mgf = "/home/eugen/Development/EuBIC2020/data/most_similar_cluster_representatives.mgf"

In [ ]:
# load clustered spectra
exp = MSExperiment()
MascotGenericFile().load(input_mgf, exp)
raw_exp = MSExperiment()
MascotGenericFile().load(input_mgf, raw_exp)

# normalize intensities for each spectrum to a 0.0 to 1.0 range
Normalizer().filterPeakMap(exp)

In [ ]:
tolerance = 0.005

# definition of distance
# XQuestScores.xCorrelationPrescore: simple, binned dot product, normalized by number of peaks.
# third parameter of xCorrelationPrescore is binsize in Da
def distance(spec1, spec2, method='xcorr'):
    if (method=='xcorr'):
        xcorr = XQuestScores().xCorrelationPrescore(spec1, spec2, tolerance)
        return 1.0-xcorr
    
    if (method=='int_diff'):
        
        # alignment of peaks with 20 ppm tolerance
        sa = SpectrumAlignment()
        param = sa.getParameters()
        param.setValue("tolerance", tolerance)
        true_string = "false"
        param.setValue("is_relative_tolerance", true_string.encode())
        sa.setParameters(param)

        alignment = []
        sa.getSpectrumAlignment(alignment, spec1, spec2)
        
        # sum up intensity differences between matched peaks
        in_spec1 = []
        in_spec2 = []
        dist = 0

        for i in range(0, len(alignment)):
            in_spec1.append(alignment[i][0])
            in_spec2.append(alignment[i][1])
            dist += abs(spec1[alignment[i][0]].getIntensity() - spec2[alignment[i][1]].getIntensity())

        for i in range(0, spec1.size()):
            if (i not in in_spec1):
                dist += spec1[i].getIntensity()
                
        for i in range(0, spec2.size()):
            if (i not in in_spec2):
                dist += spec2[i].getIntensity()
        return dist
    
    else:
        return 0

In [ ]:
cluster_names = []
cluster_membership = []

# extract cluster names from TITLE
for i in range(0,exp.size()):
    cl_name = exp[i].getMetaValue("TITLE").decode().split(";")[0]
    cluster_membership.append(cl_name)
    if cl_name not in cluster_names:
        cluster_names.append(cl_name)


In [ ]:
cluster_membership = pd.Series(cluster_membership)
cluster_names[0] == cluster_membership

In [ ]:
export_spec = MSExperiment()
range_start = 0

for cl in cluster_names:
    
    print(cl)
    # collect the spectra in the current cluster
    cluster_reached = False
    cluster_ended = False
    cluster_spec = []
    for i in range(range_start, cluster_membership.size):
        if (cl == cluster_membership[i]):
            cluster_spec.append(i)
            if (not cluster_reached): # found start of cluster
                cluster_reached = True
        else: # reached end of cluster
            if (cluster_reached):
                range_start = i-1
                break
    
    # if the cluster only contains one spectrum, just return that one
    print(len(cluster_spec))
    if (len(cluster_spec) == 1):
        export_spec.addSpectrum(raw_exp[cluster_spec[0]])
        continue
        
    # if the cluster does not have any spectra, skip it (should not happen)
    if (len(cluster_spec) == 0):
        continue
    
    # initialize distance matrix
    dist_matrix = np.zeros((len(cluster_spec), len(cluster_spec)))

    # calculate pairwise distances (fill triangular matrix)
    for i in range(0, len(cluster_spec)):
        for j in range(i, len(cluster_spec)):
            dist_matrix[i][j] = distance(exp[cluster_spec[i]], exp[cluster_spec[j]])
    
    dist_matrix = pd.DataFrame(dist_matrix)

    # summarize distances from each spetrum to all others
    total_dist = np.zeros(dist_matrix.iloc[0,:].size)
    for i in range(0, dist_matrix.iloc[0,:].size):
        total_dist[i] = (dist_matrix.iloc[i,:].sum() + dist_matrix.iloc[:,i].sum()) / dist_matrix.iloc[0,:].size

    # find best spectrum with minimal distance to all others
    best_spec = np.where(total_dist == np.amin(total_dist))

    # add best spectrum to set of exported spectra
    # first option is used, if several spectra have the same total distance
    best_spec = best_spec[0]
    if (best_spec.size > 1):
        best_spec = best_spec[0]
    best_spec = best_spec.item()
    export_spec.addSpectrum(raw_exp[cluster_spec[best_spec]])

# write out MGF file containing all representative spectra  
MascotGenericFile().store(output_mgf, export_spec)
        